In [21]:
import pandas as pd
import numpy as np
import pyprind
import os
import re

## Preprocessing the movie dataset into more convenient format

In [22]:
# change the `basepath` to the directory of the unzipped movie dataset


# basepath = 'aclImdb'
# labels = {'pos': 1, 'neg': 0}

# pbar = pyprind.ProgBar(50000)
# df = pd.DataFrame()

# for s in ('test', 'train'):
#     for l in ('pos', 'neg'):
#         path = os.path.join(basepath,s,l)
#         for file in os.listdir(path):
#             with open(os.path.join(path,file),'r',encoding = 'utf-8') as infile:
#                 txt = infile.read()
#             df = df.append([[txt, labels[l]]], ignore_index = True)
#             pbar.update()
# df.columns = ['review','sentiment']

In [23]:
# shuffle the data as it is in sorted form and then save into csv file for later use

# np.random.seed(0)
# df = df.reindex(np.random.permutation(df.index))
# df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [24]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


## Transforming words into feature vectors

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'])
bag = count.fit_transform(docs)

In [26]:
 print(count.vocabulary_)

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


In [27]:
print(bag.toarray())

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


## Accessing word relevancy via term frequency-inverse document frequency 

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer

# the tfidf results normalized score of the words based on their relevancy i.e if the word is present in the 
# multiple documents and thus is unlikely to contain any discriminatory information

tfidf = TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.56 0.56 0.   0.43 0.  ]
 [0.   0.43 0.   0.   0.56 0.43 0.56]
 [0.4  0.48 0.31 0.31 0.31 0.48 0.31]]


## Cleaning text data

In [29]:
df.loc[0,'review']

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [30]:
# using regular expression to clean the data
import re

def preprocessor(text):
    ## remove the HTML markup
    text = re.sub('<[^>]*>', '', text)
    ## find the emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    ## remove all non character words from the text and addd all the emoticons at the end of the processed string
    ## replace the '-' from the emoticons
    text = (re.sub('[\W]+', ' ', text.lower()) +' '.join(emoticons).replace('-', ''))
    return text

In [31]:
## Checkpoint

print(preprocessor(df.loc[0, 'review'][-50:]))
print(preprocessor("</a>This :) is :( a test :-)!"))

is seven title brazil not available
this is a test :) :( :)


In [32]:
## use the preprocessor to clean the review column 
df['review'] = df['review'].apply(preprocessor)

## Processing documents into tokens

In [33]:
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [34]:
## Word stemming - transforming the word into its root form 
## eg: running is transfromed as run

from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [35]:
## stop word removal - removing the words such as (is, was, like, and, has) are common and bear no useful
## useful infomation to classify the documents.

import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

stop = stopwords.words('english')

[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amitpatel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

## Training Logistic Regression model

In [36]:
## splitting the data into training and test set

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values

X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [37]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_extraction.text import TfidfVectorizer

# ## combining countvectorizer and tfid transformation
# tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)

# ## grid search parameters 
# param_grid = [{'vect__ngram_range': [(1,1)],
#                 'vect__stop_words': [stop, None],
#                 'vect__tokenizer': [tokenizer,tokenizer_porter],
#                 'clf__penalty': ['l1', 'l2'],
#                 'clf__C': [1.0, 10.0, 100.0]},
#             {'vect__ngram_range': [(1,1)],
#                 'vect__stop_words': [stop, None],
#                 'vect__tokenizer': [tokenizer,tokenizer_porter],
#                 'vect__use_idf':[False],
#                 'vect__norm':[None],
#                 'clf__penalty': ['l1', 'l2'],
#                 'clf__C': [1.0, 10.0, 100.0]}
#             ]
# ## pipeline for transformer and estimator
# lr_tfidf = Pipeline([('vect', tfidf),('clf',LogisticRegression(random_state=0))])

# gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,scoring='accuracy',cv=5, verbose=1,n_jobs=1)

# gs_lr_tfidf.fit(X_train, y_train)

## Online algorithms and out of core learning

In [38]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
    text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


In [39]:
## reads in and returns one dcument at a time

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [40]:
!ls

1st_flask_app_1
1st_flask_app_2
Creating and using the virtual environment.ipynb
Sentiment_Analysis_IMDb_Movie_Review.ipynb
aclImdb
aclImdb_v1.tar.gz
movie_data.csv
test.py
web_app


In [41]:
## Checkpoint

next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [42]:
## take a document stream from stream_docs and return a particular number of documents specified by size parameter

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [43]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None,tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

doc_stream = stream_docs(path='movie_data.csv')

In [44]:
## Now we can use the code below to start out-of-core learning

pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:35


In [45]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [46]:
## using the final 5000 documents to update our model
clf = clf.partial_fit(X_test, y_test)

## Serializing fitted scikit-learn estimators

In [47]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')

if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop,open(os.path.join(dest, 'stopwords.pkl'),'wb'),protocol=4)

pickle.dump(clf,open(os.path.join(dest, 'classifier.pkl'), 'wb'),protocol=4)

In [48]:
!pickle --version

/bin/sh: pickle: command not found


## Setting up sqlite db for data storage

In [49]:
import sqlite3
import os
if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db'\
    ' (review TEXT, sentiment INTEGER, date TEXT)')
example1 = 'I love this movie'
c.execute("INSERT INTO review_db"\
    " (review, sentiment, date) VALUES"\
    " (?, ?, DATETIME('now'))", (example1, 1))
example2 = 'I disliked this movie'

In [50]:
c.execute("INSERT INTO review_db"\
          " (review, sentiment, date) VALUES"\
        " (?, ?, DATETIME('now'))", (example2, 0))
conn.commit()
conn.close()